In [69]:
import pandas as pd
import os, sys, random
module_path = os.path.abspath(os.path.join('../src/Sentiment-Analysis-Twitter/'))
if module_path not in sys.path:
    sys.path.append(module_path)

import nltk, re
import collections

import time
import sanderstwitter02
import stanfordcorpus
import stats
from __future__ import print_function

BASE_DIR = '../src/Sentiment-Analysis-Twitter/'

In [70]:
tweets1 = sanderstwitter02.getTweetsRawData(
    BASE_DIR + 'sentiment.csv')
tweets2 = stanfordcorpus.getNormalisedTweets(
    BASE_DIR + 'stanfordcorpus/'+stanfordcorpus.FULLDATA+'.5000.norm.csv')
#random.shuffle(tweets1)
#random.shuffle(tweets2)
tweets = tweets1 + tweets2
random.shuffle( tweets )
#tweets = tweets[:100]
print('\nlen( tweets ) = '+str(len( tweets )))


len( tweets ) = 10111


In [71]:
tweets[42]

["I'm goin 2 bed #TWITTER dead tonight", 'neu', 'twitter', ['#twitter']]

In [72]:
from functools import wraps
import preprocessing

procTweets = [ (preprocessing.processAll(text, subject=subj, query=quer), sent) 
              for (text, sent, subj, quer) in tweets]

In [73]:
stemmer = nltk.stem.PorterStemmer()

all_tweets = []                                             #DATADICT: all_tweets =   [ (words, sentiment), ... ]
for (text, sentiment) in procTweets:
    words = [word if(word[0:2]=='__') else word.lower() \
                for word in text.split() \
                if len(word) >= 3]
    words = [stemmer.stem(w) for w in words]                #DATADICT: words = [ 'word1', 'word2', ... ]
    all_tweets.append((words, sentiment))

In [74]:
train_tweets = all_tweets[:10000]
test_tweets = all_tweets[10001:]

In [110]:
add_ngram_feat = 2

unigrams_fd = nltk.FreqDist()
if add_ngram_feat > 1 :
    n_grams_fd = nltk.FreqDist()

for( words, sentiment ) in train_tweets:
    words_uni = words
    unigrams_fd.update(words)

    if add_ngram_feat>=2 :
        words_bi  = [ ','.join(map(str,bg)) for bg in nltk.bigrams(words) ]
        n_grams_fd.update( words_bi )

    if add_ngram_feat>=3 :
        words_tri  = [ ','.join(map(str,tg)) for tg in nltk.trigrams(words) ]
        n_grams_fd.update( words_tri )

In [111]:
unigrams_sorted = unigrams_fd.keys()
#bigrams_sorted = nltk.FreqDist(bigrams).keys()
#trigrams_sorted = nltk.FreqDist(trigrams).keys()
if add_ngram_feat > 1 :
    print( '\nlen( n_grams ) = '+str(len( n_grams_fd )) )
    ngrams_sorted = [ k for (k,v) in n_grams_fd.items() if v>1]
    print( '\nlen( ngrams_sorted ) = '+str(len( ngrams_sorted )) )


len( n_grams ) = 64372

len( ngrams_sorted ) = 12447


In [112]:
def get_word_features(words):
    bag = {}
    words_uni = [ 'has(%s)'% ug for ug in words ]

    if( add_ngram_feat>=2 ):
        words_bi  = [ 'has(%s)'% ','.join(map(str,bg)) for bg in nltk.bigrams(words) ]
    else:
        words_bi  = []

    if( add_ngram_feat>=3 ):
        words_tri = [ 'has(%s)'% ','.join(map(str,tg)) for tg in nltk.trigrams(words) ]
    else:
        words_tri = []

    for f in words_uni+words_bi+words_tri:
        bag[f] = 1

    #bag = collections.Counter(words_uni+words_bi+words_tri)
    return bag

In [113]:
word_features = [get_word_features(word[0]) for word in train_tweets]

In [114]:
negtn_regex = re.compile( r"""(?:
        ^(?:never|no|nothing|nowhere|noone|none|not|
            havent|hasnt|hadnt|cant|couldnt|shouldnt|
            wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint
        )$
    )
    |
    n't
    """, re.X) 
def get_negation_features(words):
    INF = 0.0
    negtn = [ bool(negtn_regex.search(w)) for w in words ]

    left = [0.0] * len(words)
    prev = 0.0
    for i in range(0,len(words)):
        if( negtn[i] ):
            prev = 1.0
        left[i] = prev
        prev = max( 0.0, prev-0.1)

    right = [0.0] * len(words)
    prev = 0.0
    for i in reversed(range(0,len(words))):
        if( negtn[i] ):
            prev = 1.0
        right[i] = prev
        prev = max( 0.0, prev-0.1)

    return dict( zip(
                    ['neg_l('+w+')' for w in  words] + ['neg_r('+w+')' for w in  words],
                    left + right ) )

In [115]:
 def extract_features(words):
    features = {}

    word_features = get_word_features(words)
    features.update( word_features )

    negation_features = get_negation_features(words)
    features.update( negation_features )
    return features


In [116]:
v_train = nltk.classify.apply_features(extract_features, train_tweets)
v_test = nltk.classify.apply_features(extract_features, test_tweets)

In [117]:
v_test[2]

({'has(didnt)': 1,
  'has(didnt,write)': 1,
  'has(donald)': 1,
  'has(donald,petri)': 1,
  'has(for)': 1,
  'has(for,that)': 1,
  'has(her)': 1,
  'has(her,newest)': 1,
  'has(hope)': 1,
  'has(hope,donald)': 1,
  u'has(let)': 1,
  'has(let,hope)': 1,
  u'has(make)': 1,
  'has(make,for)': 1,
  u'has(movi)': 1,
  'has(movi,let)': 1,
  'has(newest)': 1,
  'has(newest,movi)': 1,
  'has(nia)': 1,
  'has(nia,vardalo)': 1,
  u'has(petri)': 1,
  'has(petri,make)': 1,
  'has(sad)': 1,
  'has(sad,that)': 1,
  'has(that)': 1,
  'has(that,nia)': 1,
  u'has(vardalo)': 1,
  'has(vardalo,didnt)': 1,
  'has(write)': 1,
  'has(write,her)': 1,
  'neg_l(didnt)': 1.0,
  'neg_l(donald)': 0.30000000000000016,
  'neg_l(for)': 1.3877787807814457e-16,
  'neg_l(her)': 0.8,
  'neg_l(hope)': 0.40000000000000013,
  u'neg_l(let)': 0.5000000000000001,
  u'neg_l(make)': 0.10000000000000014,
  u'neg_l(movi)': 0.6000000000000001,
  'neg_l(newest)': 0.7000000000000001,
  'neg_l(nia)': 0.0,
  u'neg_l(petri)': 0.2000000

In [118]:
model = nltk.classify.NaiveBayesClassifier.train(v_train)

In [119]:
nltk.classify.accuracy(model, v_test)

0.7454545454545455

In [120]:
model.classify(v_test[2][0])

'neg'

In [121]:
s = "I am not unhappy today"
stemmer = nltk.stem.PorterStemmer()
s = [stemmer.stem(w) for w in s.split()]

In [122]:
unigrams_fd = nltk.FreqDist()
unigrams_fd.update(s)

fs = extract_features(s)

In [123]:
model.classify(fs)

'neg'